In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/73/75/ead8e4489217835fd8004c9c89dad8217deacd7e2591340dbe249b58c29f/datasets-2.17.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for dill<0.3.9,>=0.3.0 from https://files.pythonhosted.org/packages/c9/7a/cef76fd8438a42f96db64ddaa85280485a9c395e7df3db8158cfec1eee34/dill-0.3.8-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/eb/3a/25c4aecb61a49d4415fd71d4f66a8a5b558dd44a52d7054ea9aa59ccbac1/xxhash-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for multiprocess from https:

In [5]:
from datasets import load_dataset

# Load the MMLU dataset
dataset = load_dataset("cais/mmlu","all")

# Explore the dataset
print(dataset)


Generating auxiliary_train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

DatasetDict({
    auxiliary_train: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 99842
    })
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 1531
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 285
    })
})


In [34]:
import pandas as pd
test = pd.DataFrame(dataset['test'])

In [35]:
test['Name'] = 'MMLU_test'

In [36]:
test.head()

,question,subject,choices,answer,Name
0,Which of the following best describes the bala...,high_school_government_and_politics,[Freedom of speech is protected except in cert...,3,MMLU_test
1,Which of the following statements does NOT acc...,high_school_government_and_politics,[Registered voters between the ages of 35 and ...,1,MMLU_test
2,Which of the following plays the most signific...,high_school_government_and_politics,[The geographical area in which the child grow...,1,MMLU_test
3,What power was granted to the states by the Ar...,high_school_government_and_politics,"[Coining money, Authorizing constitutional ame...",0,MMLU_test
4,The primary function of political action commi...,high_school_government_and_politics,"[contribute money to candidates for election, ...",0,MMLU_test


In [37]:
for k,v in test.iloc[10].items():
    print("The Column name:" + k)
    print("The Value:" + str(v))

The Column name:question
The Value:Which of the following people would most likely be accused of influence peddling?
The Column name:subject
The Value:high_school_government_and_politics
The Column name:choices
The Value:['A congressperson who retires to take a position teaching political science at a university', 'A former president who advises a current president on a particular foreign policy issue', 'A voter who researches the positions of all candidates in a race before choosing whom to support', 'An official who leaves the State Department to work as a paid consultant to foreign governments']
The Column name:answer
The Value:3
The Column name:Name
The Value:MMLU_test


In [38]:
# 1: Combine the question and choices into a single column 'Question'
df = test
df['Question'] = df.apply(lambda row: f"{row['question']} The options are: " + 
                          ', '.join([f"{chr(65+i)}) {choice}" for i, choice in enumerate(row['choices'])]), axis=1)

# 2: Rename the columns
df.rename(columns={'subject': 'Category', 'answer': 'Correct Answer'}, inplace=True)

# 3: Change the value of 'Correct Answer' to A, B, C, D
df['Correct Answer'] = df['Correct Answer'].apply(lambda x: chr(65 + x))

# Selecting the required columns
df_final = df[['Name', 'Category', 'Question', 'Correct Answer']]

df_final

,Name,Category,Question,Correct Answer
0,MMLU_test,high_school_government_and_politics,Which of the following best describes the bala...,D
1,MMLU_test,high_school_government_and_politics,Which of the following statements does NOT acc...,B
2,MMLU_test,high_school_government_and_politics,Which of the following plays the most signific...,B
3,MMLU_test,high_school_government_and_politics,What power was granted to the states by the Ar...,A
4,MMLU_test,high_school_government_and_politics,The primary function of political action commi...,A
...,...,...,...,...
14037,MMLU_test,prehistory,Archaeological evidence for the production of ...,B
14038,MMLU_test,prehistory,The presence of caribou bones found near the H...,B
14039,MMLU_test,prehistory,Experiments with stone spear points made to re...,A
14040,MMLU_test,prehistory,What is the date of the first evidence of the ...,B


In [39]:
df_final.to_csv('MMLU_test.csv',index = False)